# Initialization

In [ ]:
from symxplorer.designer_tools.bayesian_ax import Ax_LTspice_Bode_Fitter

from symxplorer.spice_engine.spicelib    import LTspice_Wrapper

from symxplorer.designer_tools.tf_models import Second_Order_LP_TF, First_Order_LP_TF, cascade_tf

from symxplorer.designer_tools.utils     import Frequency_Weight, plot_complex_response

# Sizing The LP filter for given AC Response

## Create a Spicelib wrapper

In [ ]:
ltspice_wrapper = LTspice_Wrapper(asc_filename="asc/TIA-ALL_Z-LPF-AC.asc", 
                          traces_of_interest=['frequency', 'Vout'],
                          dump_parent_folder="runner")

In [ ]:
ltspice_wrapper.cap_unit, ltspice_wrapper.res_unit

## Define Target TF

In [ ]:
# Butterworth Filter (3rd order)
fc = 1e7
tf1 = Second_Order_LP_TF(q=1, fc=1.272*fc, dc_gain=1e3)
tf2 = First_Order_LP_TF(fc=1*fc, dc_gain=1)

target_tf = cascade_tf(list_of_tfs=[tf1, tf2], dc_gain_multiplier=1)

target_tf

## Create Ax wrapper

In [17]:
frequency_weights = Frequency_Weight(lower= 0.001*fc, upper= 10*fc, bias=10)
# We dont know the frequency until we run the simulation
# frequency_weights.compute_weights()

Results from the symbolic sizing stage. We will use this to shrink the search space for the spice-in-the-loop sizing
{'C_1': 5.085127777248505e-12,
  'R_5': 6027.164065244698,
  'R_2': 2052.64637549578,
  'R_3': 594.0490703581795,
  'R_1': 32164.219541514758,
  'C_4': 2.7152591702343307e-11,
  'R_6': 5858.0603096601435,
  'C_6': 2.7183594035896026e-12}

In [ ]:
optimizer = Ax_LTspice_Bode_Fitter(
    ltspice_wrapper=ltspice_wrapper, 
    target_tf=target_tf,
    mag_threshold=60, # in dB
    design_params= { # -1 is a dummy bound 
        "C"  : [1, 50], # in pF
        "R1" : [1, 50],
        "R2" : [1, 50], # in kOhm
        "R3" : [0.01, 1],
        "R5" : [1, 50],
        "R6" : [1, 50],
    },
    c_range= [1, 1e4],
    r_range= [1e-2, 1e2],
    output_node="V(vout)",
    frequency_weight = frequency_weights,
    max_loss=float('inf'),
    loss_type="mse",
    loss_norm_method="min-max",
    rescale_mag=True,
    random_seed=42,
    )

In [ ]:
optimizer.parameterize(log_scale=False)
optimizer.create_experiment(num_sobol_trials=5, use_outcome_constraint=True)

In [ ]:
optimizer.optimize(num_trials=40, include_mag_loss=True, include_phase_loss=False)

## Plot the Results

### Plot the best trial

In [ ]:
best_params, best_loss = optimizer.get_best(render_trace=True, use_model_predictions=False)
best_params, best_loss

In [ ]:
optimizer.plot_solution() # defualts to the best point, else specify through trial_idx=...

In [ ]:
trial, params, loss = optimizer.ax_client.get_best_trial()
trial, params, loss

### Choose a trial to plot

In [ ]:
idx = 39
complex_response = optimizer.optimization_log[idx]["complex_response"]
optimizer.optimization_log[idx]["bode_fitting_loss"], optimizer.optimization_log[idx]["params"]


In [ ]:
optimizer.optimization_log[idx]["mag_loss"], optimizer.optimization_log[idx]["phase_loss"]

In [ ]:
plot_complex_response(optimizer.frequency_array, [complex_response, optimizer.target_complex_response], ["optimized", "Target"])

In [ ]:
optimizer.frequency_weight.lower, optimizer.frequency_weight.upper

In [ ]:
optimizer.render_contour_plot(param_x='C', param_y="R1", metric="bode_fitting_loss")

In [ ]:
optimizer.get_trials_as_df()

### Saving the optimizer

In [1]:
optimizer.spicelib_cleanup()

NameError: name 'optimizer' is not defined

In [ ]:
optimizer.save_ax("100_mse_minmax_mag_phase")

### Loading the optimizer

In [19]:
optimizer.ax_client = Ax_LTspice_Bode_Fitter.load_ax("checkpoints/100_mse_minmax_mag_phase_2025-02-11_12-35-54.json")

# Sizing the LP filter for minimizing noise